# **Back_Trainslation Process**

# **setting**

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

#import os
#os.chdir('./data')
#os.getcwd()

In [2]:
def preprocessing_dataset(dataset):
    """처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    subject_entity = []
    object_entity = []

    for i, j in tqdm(zip(dataset["subject_entity"], dataset["object_entity"]), desc="preprocessing"):
        i = i[1:-1].split(",")[0].split(":")[1]
        j = j[1:-1].split(",")[0].split(":")[1]

        subject_entity.append(i)
        object_entity.append(j)

    out_dataset = pd.DataFrame(
        {
            "id": dataset["id"],
            "sentence": dataset["sentence"],
            "subject_entity": subject_entity,
            "object_entity": object_entity,
            "label": dataset["label"],
        }
    )
    return out_dataset


def load_data(dataset_dir):
    """csv 파일을 경로에 맡게 불러 옵니다."""
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_dataset(pd_dataset)

    return dataset

def label_to_num(label):
    num_label = []
    with open("/opt/ml/code/dict_label_to_num.pkl", "rb") as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label

# **데이터 불러오기**

In [3]:
train_path = '/opt/ml/dataset/train/train.csv'
total_data = load_data(train_path)
# total_data.columns

preprocessing: 32470it [00:00, 347413.07it/s]


In [4]:
train_data = total_data.sample(frac=0.9, random_state=42)
val_data = total_data.drop(train_data.index)

train_label = label_to_num(train_data["label"].values)
val_label = label_to_num(val_data["label"].values)

# **Back Translation**

git clone https://github.com/kakaobrain/pororo.git     #라이브러리 설치   
cd pororo         # 디렉토리 이동   
pip install -e .

In [5]:
from pororo import Pororo
mt = Pororo(task="translation", lang="multi")
#mt = Pororo(task="machine_translation", lang="multi")

2022-11-30 08:28:12.495555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
pd.options.display.max_colwidth = 2000

### **Base Code**

In [7]:
def back_trans_pororo(original_text, lang):
    text_to_lang = mt(original_text, src="ko", tgt=lang)
    new_text = mt(text_to_lang, src=lang, tgt="ko")
    return new_text

## TEST

- 한-일-한 역번역
- 문법 검사기 X 

In [8]:
def kjk_mt(dataset,lang):           
    # plan01 : 역변역 진행 후, sub,obj 존재하면 return
    # plan02 : sub -> ^$&S / obj -> ^$&O 로 변환하여 masking_sentence를 만들고 역변역 진행 후, 원래 sub,obj로 변환.
    # plan03 : ''
    
    count = 0
    result = list()
    for sub,obj,sent,id in zip(dataset["subject_entity"], dataset["object_entity"],dataset["sentence"],dataset['id']):
        print("processing... [",id,"] ")
        sub = sub.strip()[1:-1]
        obj = obj.strip()[1:-1]
        # 이모지 제거(pororo)
        input_sentence = sent.replace('®',"")
        
        #print("###### 원본 문장[",id,"] : ",input_sentence)
        
        # 한-일-한
        try :
            trans_sentence = mt(mt(input_sentence, src="ko", tgt=lang), src=lang, tgt="ko") 
        except :                                                                                        #
            print(">>> ERROR >>> ",input_sentence)
            print(">>> re : ID >>> ",id)
            result.append('')
            count +=1
            continue
        
        if sub in trans_sentence and obj in trans_sentence :        # plan01
            #print("== plan01 == 역번역된 문장[",id,"] : ",trans_sentence)
            result.append(trans_sentence)
            continue
        else :                                                      # plan02
            # sub -> ^$&S / obj -> ^$&O로 변환
            masking_sentence = input_sentence.replace(sub,"^$&S").replace(obj,"^$&O")
            trans_sentence = mt(mt(masking_sentence, src="ko", tgt=lang), src=lang, tgt="ko")
            re_sentence = trans_sentence.replace("^$&S",sub).replace("^$&O",obj)
            #print(">> plan02 >> 역번역된 문장[",id,"] : ",re_sentence)
            
            if sub not in re_sentence or obj not in re_sentence :       # plan03
                #print(">> plan03 >> 역번역된 문장[",id,"] : ",re_sentence)
                result.append('')
                count +=1
            else :
                result.append(re_sentence)
                
    print("#### 완료 #### 빈값 rows : ",count)
    return result

In [9]:
#test01_train = train_data.sort_values(by=["id"], ascending=[True])
#lang = "ja"
#kjk_sentence = kjk_mt(test01_train,lang)        # list
## 8시 40 ~ 다음날 12:30

In [10]:
#df = pd.DataFrame(kjk_sentence,columns = ['re_sentence'])
#df.to_csv('/opt/ml/workspace/save_kjk_mt.csv',index=False,encoding = 'utf-8')
#### 완료 #### 빈값 rows :  3394

### 역번역 데이터 원본 데이터에 합치기

In [11]:
def reformat(dataset):
    out_dataset = pd.DataFrame(
            {
                "id": dataset["id"],
                "sentence": dataset["sentence"],
                "subject_entity": "{'word' : "+dataset["subject_entity"]+", 'start_idx': 0, 'end_idx': 0, 'type': 'None'",
                "object_entity": "{'word' : "+dataset["object_entity"]+", 'start_idx': 0, 'end_idx': 0, 'type': 'None'",
                "label": dataset["label"],
            }
    )
    return out_dataset

In [12]:
# 01. 불러오기
sentence_kjk = pd.read_csv('/opt/ml/workspace/save_kjk_mt.csv')
#sentence_kjk : list

In [13]:
# 02. 데이터 프레임 형태로 바꾸기
kjk_data =train_data.copy()
kjk_data = kjk_data.sort_values(by=["id"], ascending=[True])
kjk_data['sentence'] = sentence_kjk['re_sentence']

# 빈 값 제거
kjk_data = kjk_data.dropna()
# kjk_data
#29223 >>  23258(-5965)

# 형식 맞추기
re_kjk = reformat(kjk_data)
re_kjk

,id,sentence,subject_entity,object_entity,label
0,0,<소메팅>은 조지 해리슨가 써서비틀즈가 1969년 앨범 <AbbeyRoad>에 담긴 노래다.,"{'word' : '비틀즈', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '조지 해리슨', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",no_relation
1,1,호남 기반의 발른 미래당·대안신당·민주평화당이 우여곡절 끝에 합쳐 민생당(가칭)으로 재생한다.,"{'word' : '민주평화당', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '대안신당', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",no_relation
2,2,"K리그2에서 성적 1위를 달리고 있는 광주FC는 26일, 한국프로축구연맹으로부터 관중 유치의 성과와 마케팅 성과를 인정받아 풀스타디움상과 플라스타지움상을 수상했다.","{'word' : '광주FC', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '한국프로축구연맹', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",org:member_of
3,3,균일가의 생활용품점(주)아성다이소(대표박정부)는 코로나 19바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 신고했다고 10일 밝혔다.,"{'word' : '아성다이소', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '박정부', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",org:top_members/employees
4,4,1967년 프로야구 드래프트 1위로 요미우리 자이언츠에 입단해 등번호는 8번으로 배정되었다.,"{'word' : '요미우리 자이언츠', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '1967', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",no_relation
...,...,...,...,...,...
29216,29216,"1852년 미국 대통령 선거에서 휘그당는 미-멕시코 전쟁의 영웅 테일러 장군의 죽음으로, 총통직을 계승한 현직 대선밀러드 필모어를 후보지명으로 거절했다.","{'word' : '송제헌', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '1986년 7월 17일', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",per:date_of_birth
29218,29218,"그의 할아버지는 루이 13세이고, 남편의루이 14세는 할머니가 된다.","{'word' : '보카 주니어스', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '아르헨티나', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",org:place_of_headquarters
29219,29219,KIA 타이거즈외야수이창진가 롯데백화점의 광주점이 상하는 9월간 MVP로 선정돼 28일 광주 LG전에 앞서 상식을 치렀다.,"{'word' : '다니엘 파사레야', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '리버 플레이트', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",per:employee_of
29220,29220,"한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표와 정책위원장 후보자 등록 마감 결과, 강석호(3선·경북·영양·영덕·봉덕·울진)-이장우(재선·대전·동구),유기준(4선·부산 서구·동구)-파크 성중(초선·서울), 김선동(재선・서울 도봉을)-김정석(초선,비례), 심재철(5선·경기·안양시","{'word' : '곽영주', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : '경무대', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",per:employee_of


In [14]:
# 03 .원본 데이터 불러와서 합치기
train_path = '/opt/ml/dataset/train/train.csv'
pd_dataset = pd.read_csv(train_path)

add_kjk_train_data = pd.concat([pd_dataset,re_kjk])
add_kjk_train_data = add_kjk_train_data.drop_duplicates(keep=False, inplace=False, ignore_index=False)
#add_kjk_train_data
# # 중복없음 확인

In [15]:
# 04. 저장
#add_kjk_train_data.to_csv('/opt/ml/dataset/train/aug_kjk.csv',index=False,encoding = 'utf-8')

- 성능 확인하기